# Bulk Download Data with `earthaccess`

Whew – we’ve processed some reflectance data! But you may have noticed
that your image doesn’t include all of Denver, or all the spectral bands
you will need – Your search returned 60 different files covering
different spectral bands and spatial areas! To work with all of them, we
will have to utilize **DRY** coding techniques – `for` loops and
functions.

First things first – load your stored variables into memory:

In [1]:
%store -r denver_redlining_gdf data_dir

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Import packages</div></div><div class="callout-body-container callout-body"><p>Add imports for packages that help you:</p>
<ol type="1">
<li>Work with the file system interoperably</li>
<li>Work with vector data</li>
<li>Create interactive plots of vector data</li>
<li>Group and aggregate tabular data</li>
</ol></div></div>

In [25]:
# Interoperable file paths
import os
import pathlib
import re # Use regular expressions to extract metadata

import geopandas as gpd 
import matplotlib.pyplot as plt
import hvplot.pandas

import earthaccess # Access NASA data from the cloud
# Overlay plots
import numpy as np # Process bit-wise cloud mask
import pandas as pd
# Group and aggregate
# Work with raster data
import rioxarray as rxr
from rioxarray.merge import merge_arrays # Merge rasters
import cartopy.crs as ccrs # CRSs

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Set up `earthaccess` connection</div></div><div class="callout-body-container callout-body"><p>As you did before, log in to your <code>earthaccess</code> account,
search for the Denver data, and open the file connections. You will find
that you have to re-run your search every time you access a file – keep
this in mind if you get errors later!</p></div></div>

In [3]:
# login
earthaccess.login(strategy="interactive", persist=True)

In [4]:
# Search earthaccess

denver_results = earthaccess.search_data(
    short_name="HLSL30",
    bounding_box=tuple(denver_redlining_gdf.total_bounds),
    temporal=("2023-07-11"),
    count=1
)

hls_data = earthaccess.open(denver_results)
hls_data

QUEUEING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/15 [00:00<?, ?it/s]

[<File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T13TEE.2023192T174310.v2.0/HLS.L30.T13TEE.2023192T174310.v2.0.B05.tif>,
 <File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T13TEE.2023192T174310.v2.0/HLS.L30.T13TEE.2023192T174310.v2.0.VAA.tif>,
 <File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T13TEE.2023192T174310.v2.0/HLS.L30.T13TEE.2023192T174310.v2.0.B10.tif>,
 <File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T13TEE.2023192T174310.v2.0/HLS.L30.T13TEE.2023192T174310.v2.0.B06.tif>,
 <File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T13TEE.2023192T174310.v2.0/HLS.L30.T13TEE.2023192T174310.v2.0.B03.tif>,
 <File-like object HTTPFileSystem, https://data.lpdaac.earth

## Putting it all together

To get a complete image, we will have to:

1.  Load in all 4 rasters that cover Denver
2.  Process them
3.  Merge, or mosaic, them into one image

And…we haven’t even talked about the other bands you might need, or what
to do if you want a time-series of images. You can see that if we were
to copy and paste all the code above for each raster we need to load, it
could get pretty overwhelming, and *very* error-prone. This is the
opposite of what we mean by **DRY** (Don’t Repeat Yourself) code.

Before we continue, we’re going to make some functions to do the tasks
you completed up above. We’ve already set up the code for you,
identified **parameters** and **returns**, and added **docstrings** to
document your function. When writing a function, we recommend the
following process:

1.  Copy the code you’re using into the function shell, making sure to
    indent it so Python knows it is part of the function.
2.  Identify any variables that are **too specific**. For example, if I
    called a `DataArray` `denver_da`, but I could use my function to
    process data from anywhere…I might change the name to `da` whereever
    it appears.
3.  Identify the function **parameters** or arguments and make sure they
    match your code. Python may let you, but we recommend not using
    variables defined outside the function inside the function – if you
    need something in your function make sure to pass it in as a
    parameter!
4.  Identify the function **returns** and make sure they match your
    code. What do you want to be able to access at the end? Keep in mind
    that unless you return them, the variables you create in a function
    will be stuck there. The `return` statement at the end of your
    function will pass the variables you want to keep back out of the
    function.
5.  Write some code to test your function, such as by plotting the
    results. We recommend going line by line when you’re just getting
    started. This usually involves commenting code later on, and
    modifying the returns and test code as you go.
6.  Restart the kernel and run your function to check that doesn’t have
    any hidden requirements.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: DRY code with functions</div></div><div class="callout-body-container callout-body"><p>Take each processing step from above, and create a function to do it.
We recommend writing the following 2 functions:</p>
<ol type="1">
<li>A function to load a raster, crop it, and apply the scale
factor</li>
<li>A function to process the cloud mask</li>
</ol>
<p>Applying the cloud mask is a single line of code, so we don’t think
it needs its own function.</p>
<p>Make sure to test all your functions using your example from up
top!</p></div></div>

In [10]:
def process_image(uri, bounds_gdf):
    """
    Load, crop, and scale a raster image from earthaccess

    Parameters
    ----------
    uri: file-like or path-like
      File accessor downloaded or obtained from earthaccess
    bounds_gdf: gpd.GeoDataFrame
      Area of interest to crop to

    Returns
    -------
    cropped_da: rxr.DataArray
      Processed raster
    """
    # load and scale
    da = rxr.open_rasterio(uri, mask_and_scale=True).squeeze()

    # obtain crs from raster
    reflectance_data_crs = da.rio.crs

    # match coordinate reference systems
    bounds_gdf = bounds_gdf.to_crs(reflectance_data_crs)

    # get study bounds da
    study_bounds = bounds_gdf.total_bounds

    # crop
    cropped_da = da.rio.clip_box(
      minx=study_bounds[0], 
      miny=study_bounds[1], 
      maxx=study_bounds[2], 
      maxy=study_bounds[3]
    )

    return cropped_da

In [11]:
# Find a green layer path
green_path = hls_data[4] # B03 = green

cropped_image = process_image(green_path, denver_redlining_gdf)

In [19]:
def process_cloud_mask(cloud_uri, bits_to_mask):
    """
    Load an 8-bit Fmask file and process to a boolean mask

    Parameters
    ----------
    uri: file-like or path-like
      Fmask file accessor downloaded or obtained from earthaccess
    bits_to_mask: list of int
      The indices of the bits to mask if set

    Returns
    -------
    cloud_mask: np.array
      Cloud mask
    """
    cloud_da = rxr.open_rasterio(cloud_uri, mask_and_scale=True).squeeze()

    # Get the cloud mask as bits
    cloud_bits = (
        np.unpackbits(
            (
                # Get the cloud mask as an array...
                cloud_da.values
                # ... of 8-bit integers
                .astype('uint8')
                # With an extra axis to unpack the bits into
                [:, :, np.newaxis]
            ), 
            # List the least significat bit first to match the user guide
            bitorder='little',
            # Expand the array in a new dimension
            axis=-1)
    )
    
    # Add up the bits for each pixel
    cloud_mask = np.sum(
        # Select bits 
        cloud_bits[:,:,bits_to_mask], 
        # Sum along the bit axis
        axis=-1
    )

    # Mask the pixel if the sum is greater than 0
    # (If any of the bits are True)
    cloud_mask = cloud_mask == 0

    return cloud_mask

In [17]:
# find cloud Fmask
cloud_path = hls_data[8] # Fmask

bits_to_mask = [
    1, # Cloud
    2, # Adjacent to cloud
    3, # Cloud shadow
    5 # Water (to not confuse with greenspace)
]

In [20]:
denver_cloud_mask = process_cloud_mask(cloud_path, bits_to_mask)
denver_cloud_mask

/var/folders/t3/h1jr9k717rsd7qwky1jw1bz00000gn/T/ipykernel_1271/1944118658.py:26: RuntimeWarning: invalid value encountered in cast
  .astype('uint8')


array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

Our next new tool to help you write DRY and correct code is the regular
expression. Regular expressions are a little like the patterns we use
with `glob` that contain wildcard characters (`*`) – but, they are
**much** more powerful. With regular expressions, we can extract
different segments from a string (file name, in this case) based on
landmarks – even if those segments are not always the same length!

First – run another `earthaccess` search, this time removing the
`count=1` argument, or setting it to `count=-1`. This will include all
the results.

In [21]:
# Search earthaccess
denver_all_results = earthaccess.search_data(
    short_name="HLSL30",
    bounding_box=tuple(denver_redlining_gdf.total_bounds),
    temporal=("2023-07-11"),
    count=-1
)

# Open all earthaccess results
all_hls_data = earthaccess.open(denver_all_results)
all_hls_data

QUEUEING TASKS | :   0%|          | 0/7260 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/7260 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/7260 [00:00<?, ?it/s]

[<File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T13TEE.2023192T174310.v2.0/HLS.L30.T13TEE.2023192T174310.v2.0.B05.tif>,
 <File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T13TEE.2023192T174310.v2.0/HLS.L30.T13TEE.2023192T174310.v2.0.VAA.tif>,
 <File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T13TEE.2023192T174310.v2.0/HLS.L30.T13TEE.2023192T174310.v2.0.B10.tif>,
 <File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T13TEE.2023192T174310.v2.0/HLS.L30.T13TEE.2023192T174310.v2.0.B06.tif>,
 <File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T13TEE.2023192T174310.v2.0/HLS.L30.T13TEE.2023192T174310.v2.0.B03.tif>,
 <File-like object HTTPFileSystem, https://data.lpdaac.earth

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Get metadata with **regular expressions**</div></div><div class="callout-body-container callout-body"><p>Using the code below as a starting point, extract metadata from file
names and put them into a <code>DataFrame</code>. This strategy will
help you later on because you will be able to group rasters by their
metadata values, such as tile ID, band ID, and/or date.</p>
<ol type="1">
<li>Build your regular expression. ChatGPT is a great tool to get
started with your regular expression. You can also check out <a
href="https://regex101.com/">https://regex101.com/</a> to test your
regular expressions, making sure to select the <code>Python</code>
regular expression engine.</li>
<li>Replace file_name with a <strong>string</strong> version of the URI.
You can access it from the object you got from <code>earthaccess</code>
through the <code>.full_name</code> attribute.</li>
<li>Add the URIs from <code>earthaccess</code> to the
<code>DataFrame</code> you created as a new column.</li>
</ol></div></div>

In [110]:
# Compile a regular expression to search for metadata
# Use () to select the file name ending with .tif and only output that group
uri_re = re.compile(
    r"v2.0/(HLS.L30.*.tif)"
)

# Find all the metadata in the file name 
uri_groups = [] # for uri in all_hls_data
cloud_idx = None

for idx, uri in enumerate(all_hls_data):
    if (hasattr(uri, 'full_name')):
        file_name = uri_re.findall(uri.full_name)[0]
        uri_groups.append({
            'tile_id': file_name,
            'tile_uri': uri
        })
        # grab cloud mask index
        if "Fmask" in file_name:
            cloud_idx = idx

# Create a DataFrame with the File-like URI
raster_df = pd.DataFrame(data=uri_groups, columns=['tile_id', 'tile_uri'])

# Check the results
raster_df.head()

,tile_id,tile_uri
0,HLS.L30.T13TEE.2023192T174310.v2.0.B05.tif,"<File-like object HTTPFileSystem, https://data..."
1,HLS.L30.T13TEE.2023192T174310.v2.0.VAA.tif,"<File-like object HTTPFileSystem, https://data..."
2,HLS.L30.T13TEE.2023192T174310.v2.0.B10.tif,"<File-like object HTTPFileSystem, https://data..."
3,HLS.L30.T13TEE.2023192T174310.v2.0.B06.tif,"<File-like object HTTPFileSystem, https://data..."
4,HLS.L30.T13TEE.2023192T174310.v2.0.B03.tif,"<File-like object HTTPFileSystem, https://data..."


Now you are ready to run your code repeatedly on each raster you want to
load. To do this, we’ll use a structure called a `for` loop, which runs
the same code repeatedly with different variable values. The values that
change are special variables called **looping variables**.

To set up a `for` loop, you can use the following process:

1.  Copy the code you’re using into the `for` loop shell, making sure to
    indent it so Python knows it is part of the loop.
2.  Identify any variables that are **too specific**. For example, if I
    called a `DataArray` `green_da`, but the loop will be processing
    data from other bands…I might change the name to `da` whereever it
    appears.
3.  Identify the **looping variable(s)** and make sure they match your
    code.
4.  Establish an **accumulator** – a data structure to store the result.
    Add what you want to keep from each iteration of the loop to it.
5.  Write some code to test your loop, such as by printing out an
    intermediate value in the loop, or eventually plotting the final
    results. We recommend going line by line when you’re just getting
    started. This usually involves commenting code later on, and
    modifying the accumulator and testing as you go. You can also use
    the `break` keywork to stop the loop after a single iteration for
    testing.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Process data</div></div><div class="callout-body-container callout-body"><p>Process all your bands. We’ve provided the structure of the
<code>for</code> loop – you will need to call your functions and work
out how to pass them the arguments they need.</p>
<p>Something that is tricky about looping through
<code>DataFrame</code>s is that they tend to wrap values in external
structures like <code>Series</code> (which are the data type of columns
in a <code>DataFrame</code>). The print out of a <code>Series</code>
containing one value and the print out of that value are not identical,
but the are <strong>very similar</strong>. To get a
<strong>value</strong> out from inside a <code>Series</code>, you can
add the following code: <code>.values[0]</code>. This will first remove
the <code>Series</code> wrapper, leaving an array, and the get the first
value in the array.</p></div></div>

In [102]:
head2 = raster_df.head(2)
head2

,tile_id,tile_uri
0,HLS.L30.T13TEE.2023192T174310.v2.0.B05.tif,"<File-like object HTTPFileSystem, https://data..."
1,HLS.L30.T13TEE.2023192T174310.v2.0.VAA.tif,"<File-like object HTTPFileSystem, https://data..."


In [104]:
for index, row in head2.iterrows():
    print(row.tile_id)

    tile_da = rxr.open_rasterio(row.tile_uri, mask_and_scale=True).squeeze()
    display(tile_da)

    # cloud_mask = process_cloud_mask(cloud_path, bits_to_mask) # cloud_path is coming from raster_df.Filename
    # cloud_path = glob(os.path.join(hls_dir, '*Fmask*.tif'))[0]
    # cloud_da = rxr.open_rasterio(cloud_path, mask_and_scale=True).squeeze()
    # cloud_da.plot()

HLS.L30.T13TEE.2023192T174310.v2.0.B05.tif


<xarray.DataArray (y: 3660, x: 3660)> Size: 54MB
[13395600 values with dtype=float32]
Coordinates:
    band         int64 8B 1
  * x            (x) float64 29kB 5e+05 5e+05 5.001e+05 ... 6.097e+05 6.098e+05
  * y            (y) float64 29kB 4.5e+06 4.5e+06 4.5e+06 ... 4.39e+06 4.39e+06
    spatial_ref  int64 8B 0
Attributes: (12/33)
    ACCODE:                    Lasrc
    arop_ave_xshift(meters):   0
    arop_ave_yshift(meters):   0
    arop_ncp:                  0
    arop_rmse(meters):         0
    arop_s2_refimg:            NONE
    ...                        ...
    TIRS_SSM_MODEL:            FINAL
    TIRS_SSM_POSITION_STATUS:  ESTIMATED
    ULX:                       499980
    ULY:                       4500000
    USGS_SOFTWARE:             LPGS_16.3.0
    AREA_OR_POINT:             Area

HLS.L30.T13TEE.2023192T174310.v2.0.VAA.tif


<xarray.DataArray (y: 3660, x: 3660)> Size: 54MB
[13395600 values with dtype=float32]
Coordinates:
    band         int64 8B 1
  * x            (x) float64 29kB 5e+05 5e+05 5.001e+05 ... 6.097e+05 6.098e+05
  * y            (y) float64 29kB 4.5e+06 4.5e+06 4.5e+06 ... 4.39e+06 4.39e+06
    spatial_ref  int64 8B 0
Attributes:
    L1T_SceneID:         2023-07-11_034032_SZA.img
    OVR_RESAMPLING_ALG:  NEAREST
    AREA_OR_POINT:       Area

In [14]:
# Refer to this earth data science textbook example on how you will merge and apply the cloud masks
# It also contains example for band processing. band = .tif file
# https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/landsat-in-Python/remove-clouds-from-landsat-data/

# May also check this one too: https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/landsat-in-Python/replace-raster-cell-values-in-remote-sensing-images-in-python/

# Labels for each band to process
bands = {
    'B01': 'aerosol',
    'B02': 'red',
    'B03': 'green',
    'B04': 'blue',
    'B05': 'nir',
    'B06': 'swir1',
    'B07': 'swir2',
    'B09': 'cirrus',
    'B10': 'thermalir1',
    'B11': 'thermalir2'
}
# Initialize structure for saving images
denver_das = {band_name: [] for band_name in bands.values()}

for tile_id, tile_df in raster_df.groupby('tile_id'):
    # Load the cloud mask
    cloud_mask = process_cloud_mask(cloud_path, bits_to_mask) # cloud_path is coming from raster_df.Filename

    for band_id, row in tile_df.groupby('band_id'):
        if band_id in bands:
            band_name = bands[band_id]
            # Process band
            
            # Mask band
            
            
            # Store the resulting DataArray ofr later
            denver_das[band_name].append(band_masked_da)

# Merge all tiles
denver_das = {
    band_name: merge_arrays(das) 
    for band_name, das in denver_das.items()
}

denver_das['green'].plot(cmap='Greens', robust=True)

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Check your data</div></div><div class="callout-body-container callout-body"><p>Make a plot of one of your merged bands with the denver boundary
superimposed. You should now have data for the whole city!</p></div></div>

In [16]:
# Plot a merged raster band
